In [1]:
from torchHHT import hht
import torch, os
import pandas as pd

In [2]:
%%time
fs = 25600
directory = "data/FEMTO/Bearing1_3"

for fileName in os.listdir(directory):
    filePath = os.path.join(directory, fileName)

    if(filePath[22:26] == 'temp'): #skip all temperature data
        continue

    col_names = ['Hour', 'Minute', 'Second', 'Nano_second', 'Horizontal', 'Vertical']

    data = pd.read_csv(filePath, sep=',', names=col_names) # femto1_7 dataset is seperated by comma others by semicolon
    
    horizontal = data['Horizontal']
    vertical = data['Vertical']

    tensor_horizontal = torch.Tensor(horizontal.values)
    tensor_vertical = torch.Tensor(vertical.values)

    tensor_combined = torch.sqrt(torch.pow(tensor_horizontal, 2) + torch.pow(tensor_vertical, 2))


    if torch.cuda.is_available():
        device = torch.device("cuda:0")
        tensor_horizontal = tensor_horizontal.cuda()
        tensor_vertical = tensor_vertical.cuda()
        tensor_combined = tensor_combined.cuda()

    # imfs_horizontal, imfs_env_horizontal, imfs_freq_horizontal = hht.hilbert_huang(tensor_horizontal, fs, num_imf=3)
    # imfs_vertical, imfs_env_vertical, imfs_freq_vertical = hht.hilbert_huang(tensor_vertical, fs, num_imf=3)
    imfs_combined, imfs_env_combined, imfs_freq_combined = hht.hilbert_huang(tensor_combined, fs, num_imf=3)

    # imf_string_horizontal = "imfs_Bearing1_7_horizontal_" + fileName[:-4]  + ".pt"
    # imf_env_string_horizontal = "imfs_env_Bearing1_7_horizontal_" + fileName[:-4]  + ".pt"
    # imf_freq_string_horizontal = "imfs_freq_Bearing1_7_horizontal_" + fileName[:-4]  + ".pt"
    # imf_string_vertical = "imfs_Bearing1_7_vertical_" + fileName[:-4]  + ".pt"
    # imf_env_string_vertical = "imfs_env_Bearing1_7_vertical_" + fileName[:-4]  + ".pt"
    # imf_freq_string_vertical = "imfs_freq_Bearing1_7_vertical_" + fileName[:-4]  + ".pt"

    imf_string_combined = "imfs_" + filePath[11:21] + "_combined_" + fileName[:-4]  + ".pt"
    imf_env_string_combined = "imfs_env_" + filePath[11:21] + "_combined_" + fileName[:-4]  + ".pt"
    imf_freq_string_combined = "imfs_freq_" + filePath[11:21] + "_combined_" + fileName[:-4]  + ".pt"

    # print(imf_freq_string_combined)
    # break
    
    # torch.save(imfs_horizontal, os.path.join("IMFs/FEMTO/Bearing1_7/horizontal", imf_string_horizontal))
    # torch.save(imfs_vertical, os.path.join("IMFs/FEMTO/Bearing1_7/vertical", imf_string_vertical))
    torch.save(imfs_combined, os.path.join("IMFs/FEMTO/Bearing1_3/combined", imf_string_combined))

    # torch.save(imfs_env_horizontal, os.path.join("IMF_ENVs/FEMTO/Bearing1_7/horizontal", imf_env_string_horizontal))
    # torch.save(imfs_env_vertical, os.path.join("IMF_ENVs/FEMTO/Bearing1_7/vertical", imf_env_string_vertical))
    torch.save(imfs_env_combined, os.path.join("IMF_ENVs/FEMTO/Bearing1_3/combined", imf_env_string_combined))

    
    # torch.save(imfs_freq_horizontal, os.path.join("IMF_FREQs/FEMTO/Bearing1_7/horizontal", imf_freq_string_horizontal))
    # torch.save(imfs_freq_vertical, os.path.join("IMF_FREQs/FEMTO/Bearing1_7/vertical", imf_freq_string_vertical))
    torch.save(imfs_freq_combined, os.path.join("IMF_FREQs/FEMTO/Bearing1_3/combined", imf_freq_string_combined))

CPU times: user 1h 22min 46s, sys: 4min 37s, total: 1h 27min 23s
Wall time: 5min 27s
